## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\1010h\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Modelos

In [4]:
# Diccionario de modelos
MODELOS = {
    "sentence_similarity": 'hiiamsid/sentence_similarity_spanish_es',
    "sentence-transformers 1": 'uzairnaeem/finetuned_sentence_similarity_spanish_es',
    "sentence-transformers 2": "mrm8488/modernbert-embed-base-ft-sts-spanish-matryoshka-768-64-5e",
    "sentence-transformers 3" : 'Santp98/SBERT-pairs-bert-base-spanish-wwm-cased'
}

## Funciones (?

In [ ]:
CAMPOS = ["Titulo", "resumen", "objetivos", "claves"]

df = pd.read_csv("protocolos_completo_limpios.csv")

while True:
    query = input("Ingresa el texto para comparar: ").strip()
    if not query:
        print("Consulta vacía. Por favor, ingresa un texto.")
        continue
    print("\nConsulta: ", query)
    break

# Recorremos cada modelo
for clave_modelo, nombre_modelo in MODELOS.items():
    print(f"\n===== Comparando con modelo: {clave_modelo} =====")
    modelo = SentenceTransformer(nombre_modelo)

    df_temp = df.copy()

    # Recorremos cada campo
    for campo in CAMPOS:
        nombre_pkl = f"{campo}_{clave_modelo}_embeddings.pkl"
        ruta_pkl = os.path.join("PKL", nombre_pkl)

        if not os.path.exists(ruta_pkl):
            print(f"No se encontró el archivo pkl para campo '{campo}' y modelo '{clave_modelo}': {ruta_pkl}")
            continue

        with open(ruta_pkl, "rb") as f:
            embeddings_cargados = pickle.load(f)

        embedding_query = modelo.encode(query, convert_to_tensor=False)
        simi_coseno = cosine_similarity([embedding_query], embeddings_cargados)[0]
        df_temp[f"similitud_{campo}"] = simi_coseno

        df_temp[campo] = df_temp[campo].apply(
            lambda x: x[:190] + "..." if isinstance(x, str) and len(x) > 190 else x
        )

    # Verificamos que todas las columnas de similitud existan
    columnas_similitud = [f"similitud_{campo}" for campo in CAMPOS if f"similitud_{campo}" in df_temp.columns]

    if not columnas_similitud:
        print("No se encontraron similitudes para ningún campo.")
        continue

    # Sumar las similitudes de los campos
    df_temp["similitud_total"] = df_temp[columnas_similitud].sum(axis=1)
    df_temp = df_temp.sort_values(by="similitud_total", ascending=False).head(20)

    # Mostrar resultados
    columnas_mostrar = ["TT"] + CAMPOS + columnas_similitud + ["similitud_total"]
    print(df_temp[columnas_mostrar].to_string(index=False))



Consulta:  museos

===== Comparando con modelo: sentence_similarity =====
       TT                                                                                                                                                       Titulo                                                                                                                                                                                           resumen                                                                                                                                                                                         objetivos                                                                                                                          claves  similitud_Titulo  similitud_resumen  similitud_objetivos  similitud_claves  similitud_total
2020-B057                                                    AplicaciÃ³n web para difusiÃ³n de exposiciones y eventos culturales de los museos de la 

In [25]:
# Solicitar consulta al usuario
while True:
    query = input("Ingresa el texto para comparar: ").strip()
    if not query:
        print("Consulta vacía. Por favor, ingresa un texto.")
        continue
    print("\nConsulta: ", query)
    break

# Lista para guardar nuevos resultados
filas_tabla = []

# Comparar con cada modelo
for clave_modelo, nombre_modelo in MODELOS.items():
    print(f"Procesando modelo: {clave_modelo}")
    modelo = SentenceTransformer(nombre_modelo)
    df_temp = df.copy()

    for campo in CAMPOS:
        nombre_pkl = f"{campo}_{clave_modelo}_embeddings.pkl"
        ruta_pkl = os.path.join("PKL", nombre_pkl)

        if not os.path.exists(ruta_pkl):
            print(f"No se encontró el archivo {ruta_pkl}")
            continue

        with open(ruta_pkl, "rb") as f:
            embeddings_cargados = pickle.load(f)

        embedding_query = modelo.encode(query, convert_to_tensor=False)
        sim_coseno = cosine_similarity([embedding_query], embeddings_cargados)[0]
        df_temp[f"similitud_{campo}"] = sim_coseno

    columnas_sim = [f"similitud_{c}" for c in CAMPOS if f"similitud_{c}" in df_temp.columns]
    if not columnas_sim:
        continue

    df_temp["similitud_total"] = df_temp[columnas_sim].sum(axis=1)
    df_top = df_temp.sort_values("similitud_total", ascending=False).head(5)

    for i, (_, fila) in enumerate(df_top.iterrows(), start=1):
        for campo in CAMPOS:
            texto = fila[campo]
            fila_resultado = {
                "Modelo": clave_modelo,
                "Consulta": query if campo == "Titulo" and i == 1 else "",
                "No. Consulta": i,
                "Campo": campo,
                "Texto": texto,
                "Similitud": fila.get(f"similitud_" + campo, 0),
                "Justificación": ""
            }
            filas_tabla.append(fila_resultado)

        filas_tabla.append({
            "Modelo": clave_modelo,
            "Consulta": "",
            "No. Consulta": i,
            "Campo": "TOTAL",
            "Texto": "",
            "Similitud": fila["similitud_total"],
            "Justificación": ""
        })

# Crear DataFrame con nuevos resultados
df_nuevos = pd.DataFrame(filas_tabla)

# Eliminar duplicados de nombres de columnas si los hay
df_nuevos = df_nuevos.loc[:, ~df_nuevos.columns.duplicated()]

# Nombre del archivo de salida
nombre_archivo = "tabla_consultas_formato_deseado.csv"

# Cargar datos existentes si el archivo ya existe
if os.path.exists(nombre_archivo):
    df_existente = pd.read_csv(nombre_archivo)
    # Asegurar mismo orden de columnas al unir
    df_nuevos = df_nuevos[df_existente.columns]
    df_final = pd.concat([df_existente, df_nuevos], ignore_index=True)
else:
    df_final = df_nuevos

# Guardar resultados finales
df_final.to_csv(nombre_archivo, index=False)
print(f"✅ Resultados guardados en '{nombre_archivo}' sin columnas duplicadas.")


Consulta:  prueba x de n
Procesando modelo: sentence_similarity
Procesando modelo: sentence-transformers 1
Procesando modelo: sentence-transformers 2
Procesando modelo: sentence-transformers 3
✅ Resultados guardados en 'tabla_consultas_formato_deseado.csv' sin columnas duplicadas.
